In [1]:
import pickle
import numpy as np
from procedures.preprocess_files import _preprocess_data
from preprocessing.tracking import calculate_movement_to_body_ratio
from shared.structs import SkeletonData
from shared.visualize_skeleton_file import visualize
from tqdm import tqdm

In [2]:
mutual = [x for x in range(50, 61)]

In [3]:
with open("/home/barny/MasterThesis/Data/ntu_coco.2.combined", "rb") as f:
    data = pickle.load(f)
data['actions'] = [x['info']['action'] for x in data['labels']]
data['body_count'] = [x.shape[0] for x in data['points']]
data_list = zip(*data.values())

In [4]:
single_but_mutual = [x for x in data_list if x[2] not in mutual and x[3] == 2]
mutual_but_single = [x for x in data_list if x[2] in mutual and x[3] == 1]

In [5]:
print(len(single_but_mutual), len(mutual_but_single))

403 0


In [6]:
def to_ntu_filename(info):
    # Ntu would have "set", "camera", "person", "replication", "action"
    template = "S{:03d}C{:03d}P{:03d}R{:03d}A{:03d}"
    info_list = [info.get(key, 0) for key in ["set", "camera", "person", "replication", "action"]]
    return template.format(*info_list)

In [7]:
for i, data in enumerate(single_but_mutual):
    continue
    filename = to_ntu_filename(data[0]['info'])
    filepath = f"/home/barny/MasterThesis/Data/alphapose_skeletons/ntu_coco/{filename}.coco17.apskel.pkl"
    skeleton_data = SkeletonData.load(filepath)
    #visualize(data, data.video_file, wait_key=1000 // 30)
    _preprocess_data(skeleton_data, None)
    #visualize(skeleton_data, skeleton_data.video_file, wait_key=1000 // 90, window_name=f"{i} {filename}")


In [8]:
with open("/home/barny/MasterThesis/Data/ntu_coco.notfill.combined", "rb") as f:
    data = pickle.load(f)
data['actions'] = [x['info']['action'] for x in data['labels']]
data['body_count'] = [len(x.get_all_tids()) for x in data['points']]
data_list = [x for x in zip(*data.values())]

In [9]:
single_but_mutual = [x for x in data_list if x[2] not in mutual and x[3] == 2]
mutual_but_single = [x for x in data_list if x[2] in mutual and x[3] == 1]

print(len(single_but_mutual), len(mutual_but_single))

403 16


In [14]:
all_ratios = []
bad_fi = []
for fi, data in tqdm(enumerate(data_list)):
    if fi not in fis:
        continue
    info, skdata,  action, bc = data
    skdata: SkeletonData
    for tid in skdata.get_all_tids():
        bodies = skdata.get_all_bodies_for_tid(tid)
        bodies = []
        for frame in skdata.frames:
            for body in frame.bodies:
                if body.tid == tid:
                    bodies.append((frame.seqId, body.poseXY))
        ratios = []
        for i in range(len(bodies)-1):
            ratio = calculate_movement_to_body_ratio(bodies[i][1], bodies[i+1][1], bodies[i+1][0]-bodies[i][0])
            ratios.append(ratio)
            if (ratio > np.array([1.8, 0.5])).any():
                bad_fi.append((ratio, data,i))
                continue
                visualize(skdata, skdata.video_file, wait_key=1000//30, window_name=f"{fi} {ratio} visualization")
                filename = to_ntu_filename(data[0]['info'])
                filepath = f"/home/barny/MasterThesis/Data/alphapose_skeletons/ntu_coco/{filename}.coco17.apskel.pkl"
                skeleton_data = SkeletonData.load(filepath)
                #visualize(data, data.video_file, wait_key=1000 // 30)
                _preprocess_data(skeleton_data, None)
                visualize(skeleton_data, skeleton_data.video_file, wait_key=0, window_name=f"{fi} {filename}")
        all_ratios.extend(ratios)


56880it [00:00, 587307.67it/s]


In [ ]:
print(len(bad_fi))
bad_fi_done = []
for ratio, data, fi in tqdm(bad_fi):
    skdata:SkeletonData = data[1]
    tqdm.write(str(ratio))
    if skdata.dataset_info.to_filename() in bad_fi_done:
        continue
    bad_fi_done.append(skdata.dataset_info.to_filename())
    
    visualize(skdata, skdata.video_file, wait_key=0, window_name=f"{fi} {ratio} {skdata.dataset_info.to_filename()}")

17


  0%|          | 0/17 [00:00<?, ?it/s]

[0.03196525 0.50343803]


QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is n

[3.00425796 0.10047865]


...

[9.02833122 1.95721126]


 76%|███████▋  | 13/17 [18:04<03:03, 45.76s/it]QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject:

[4.62008315 0.80707887]
[8.77600056 1.78635605]
[10.78707766  0.45358391]


 94%|█████████▍| 16/17 [18:10<00:21, 21.05s/it]QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject::moveToThread: Current thread (0x5c1f3000) is not the object's thread (0x449ec3b0).
Cannot move to target thread (0x5c1f3000)

QObject:

[5.77610481 2.06892318]


In [ ]:
visualize(skeleton_data, skeleton_data.video_file, wait_key=0, window_name=f"{fi} {filename}")

In [ ]:
for frame in skeleton_data.frames:
    print([body.tid for body in frame.bodies])

In [ ]:
x = [x[0] for x in all_ratios]
y = [x[1] for x in all_ratios]
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.scatter(x,y,s=1)
plt.show()

In [12]:
fis = set([x[2] for x in bad_fi])

{7109,
 7471,
 35372,
 35376,
 35410,
 35413,
 35499,
 42756,
 43317,
 43535,
 43658,
 43890,
 50988,
 56443}